## 1. Get data

In [1]:
import DBConn as dbc
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
db = dbc.DBConn('config/silnaewichi-2c66f-firebase-adminsdk-t6ob2-a6dfec601d.json')
key = 'it5'

In [3]:
bbsid_list = []
ssid_list = []

def getFrame(data:dict, x:float, y:float):
    global bbsid_list, ssid_list
    dict_frame = {
        'x': [x for _ in range(data['n'])],
        'y': [y for _ in range(data['n'])],
        'magnetic_x': [],
        'magnetic_y': [],
        'magnetic_z': []
    }
    
    mag_tag = "magnetic"
    wifi_tag = "wifi"
    bssid_tag = "BSSID"
    
    for _, obj in data.items():
        if _ == 'n':
            continue
        # magnetic
        for dim, d in obj[mag_tag].items():
            dict_frame[f"{mag_tag}_{dim}"].append(d)
        # wifi list setting
        for _, wifi in obj[wifi_tag].items():
            if wifi['BSSID'] not in bbsid_list:
                bbsid_list.append(wifi[bssid_tag])
                ssid_list.append(wifi['SSID'])
                
    # wifi
    for bbsid in bbsid_list:
        dict_frame[bbsid] = []
    for _, obj in data.items():
        if _ == 'n':
            continue
        none_flag = {bb:False for bb in bbsid_list}
        for _, wifi in obj[wifi_tag].items():
            dict_frame[wifi[bssid_tag]].append(wifi['WifiLevel'])
            none_flag[wifi[bssid_tag]] = True
        for bb, right in none_flag.items():
            if right is False:
                dict_frame[bb].append(None)
            
    result_df = pd.DataFrame(data=dict_frame, dtype=np.float64)
    return result_df


* x: 0 ~ 12
* y:-8 ~ 8
* x break points : 0, 10, 12
* y break points : 0, 3, 8

In [4]:
%%time
all_f = []

for y in [0, 3, 8]:
    for x in range(1, 10):
        _d = db.getData(key, x, y)
        _f = getFrame(_d, x, y)
        all_f.append(_f)
        
for x in [0, 10]:
    for y in range(0, 9):
        _d = db.getData(key, x, y)
        _f = getFrame(_d, x, y)
        all_f.append(_f)
        
y = 0
for x in [11, 12]:
    _d = db.getData(key, x, y)
    _f = getFrame(_d, x, y)
    all_f.append(_f)
    
x = 12
for y in range(-8, 0):
    _d = db.getData(key, x, y)
    _f = getFrame(_d, x, y)
    all_f.append(_f)

CPU times: user 16.3 s, sys: 2.98 s, total: 19.2 s
Wall time: 4min 56s


In [5]:
dict_frame = {
    'x': [],
    'y': [],
    'magnetic_x': [],
    'magnetic_y': [],
    'magnetic_z': []
}
for bss in bbsid_list:
    dict_frame[bss] = []
    
result_df = pd.DataFrame(data=dict_frame)
for _df in all_f:
    result_df = result_df.append(_df, ignore_index=True)
result_df.head()
result_df.shape

/home/gwak/anaconda3/envs/cpl/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(315, 34)

In [6]:
result_df.dtypes

08:d4:6a:e6:ff:39    float64
28:c2:dd:70:6f:51    float64
28:c2:dd:70:70:4b    float64
28:c2:dd:70:70:4f    float64
28:c2:dd:70:70:51    float64
28:c2:dd:70:93:0d    float64
68:ca:e4:46:c1:80    float64
68:ca:e4:46:c1:81    float64
68:ca:e4:47:4d:e0    float64
68:ca:e4:47:4d:e1    float64
68:ca:e4:47:4d:ee    float64
68:ca:e4:47:4d:ef    float64
68:ca:e4:47:4e:a0    float64
68:ca:e4:47:4e:a1    float64
68:ca:e4:47:51:80    float64
68:ca:e4:47:51:81    float64
68:ca:e4:47:51:8e    float64
68:ca:e4:47:51:8f    float64
68:ca:e4:47:54:21    float64
68:ca:e4:47:6e:e0    float64
68:ca:e4:48:9d:a0    float64
68:ca:e4:48:9d:a1    float64
68:ca:e4:48:9d:ae    float64
68:ca:e4:48:9d:af    float64
88:36:6c:08:80:8c    float64
88:36:6c:08:80:8e    float64
88:36:6c:a1:47:7a    float64
88:36:6c:bb:10:06    float64
90:9f:33:a8:6b:b4    float64
magnetic_x           float64
magnetic_y           float64
magnetic_z           float64
x                    float64
y                    float64
dtype: object

In [7]:
result_df.isnull().sum()

08:d4:6a:e6:ff:39     18
28:c2:dd:70:6f:51      0
28:c2:dd:70:70:4b      0
28:c2:dd:70:70:4f      0
28:c2:dd:70:70:51      0
28:c2:dd:70:93:0d     18
68:ca:e4:46:c1:80    306
68:ca:e4:46:c1:81    306
68:ca:e4:47:4d:e0    228
68:ca:e4:47:4d:e1    233
68:ca:e4:47:4d:ee    212
68:ca:e4:47:4d:ef    207
68:ca:e4:47:4e:a0    313
68:ca:e4:47:4e:a1    306
68:ca:e4:47:51:80      0
68:ca:e4:47:51:81      0
68:ca:e4:47:51:8e     18
68:ca:e4:47:51:8f     18
68:ca:e4:47:54:21    292
68:ca:e4:47:6e:e0    297
68:ca:e4:48:9d:a0     75
68:ca:e4:48:9d:a1     71
68:ca:e4:48:9d:ae    189
68:ca:e4:48:9d:af    172
88:36:6c:08:80:8c    145
88:36:6c:08:80:8e    141
88:36:6c:a1:47:7a    254
88:36:6c:bb:10:06     42
90:9f:33:a8:6b:b4    279
magnetic_x             0
magnetic_y             0
magnetic_z             0
x                      0
y                      0
dtype: int64

In [8]:
result_df.to_csv("./data/full_data.csv")

In [13]:
bssid_ssid = dict()
for i, (b, n) in enumerate(zip(bbsid_list, ssid_list)):
    bssid_ssid[b] = n
pprint(bssid_ssid)

{'08:d4:6a:e6:ff:39': 'Q8(2018)_7622',
 '28:c2:dd:70:6f:51': 'SCD-696013949W_FB',
 '28:c2:dd:70:70:4b': 'SCD-696013969W_54',
 '28:c2:dd:70:70:4f': 'SCD-696013899W_44',
 '28:c2:dd:70:70:51': 'SCD-696013929W_XN',
 '28:c2:dd:70:93:0d': 'SCD-696013979W_QS',
 '68:ca:e4:46:c1:80': 'knu-smart',
 '68:ca:e4:46:c1:81': 'eduroam',
 '68:ca:e4:47:4d:e0': 'knu-smart',
 '68:ca:e4:47:4d:e1': 'eduroam',
 '68:ca:e4:47:4d:ee': 'eduroam',
 '68:ca:e4:47:4d:ef': 'knu-smart',
 '68:ca:e4:47:4e:a0': 'knu-smart',
 '68:ca:e4:47:4e:a1': 'eduroam',
 '68:ca:e4:47:51:80': 'knu-smart',
 '68:ca:e4:47:51:81': 'eduroam',
 '68:ca:e4:47:51:8e': 'eduroam',
 '68:ca:e4:47:51:8f': 'knu-smart',
 '68:ca:e4:47:54:21': 'eduroam',
 '68:ca:e4:47:6e:e0': 'knu-smart',
 '68:ca:e4:48:9d:a0': 'knu-smart',
 '68:ca:e4:48:9d:a1': 'eduroam',
 '68:ca:e4:48:9d:ae': 'eduroam',
 '68:ca:e4:48:9d:af': 'knu-smart',
 '88:36:6c:08:80:8c': 'DBLAB_5G',
 '88:36:6c:08:80:8e': 'DBLAB',
 '88:36:6c:a1:47:7a': 'NetLab',
 '88:36:6c:bb:10:06': 'iptime',
 '90:

## 2. data cleansing & fillna

In [12]:
cut_df = result_df.copy()
cut_df.shape

(315, 34)

In [25]:
result_df.isnull().sum() > 30

08:d4:6a:e6:ff:39    False
28:c2:dd:70:6f:51    False
28:c2:dd:70:70:4b    False
28:c2:dd:70:70:4f    False
28:c2:dd:70:70:51    False
28:c2:dd:70:93:0d    False
68:ca:e4:46:c1:80     True
68:ca:e4:46:c1:81     True
68:ca:e4:47:4d:e0     True
68:ca:e4:47:4d:e1     True
68:ca:e4:47:4d:ee     True
68:ca:e4:47:4d:ef     True
68:ca:e4:47:4e:a0     True
68:ca:e4:47:4e:a1     True
68:ca:e4:47:51:80    False
68:ca:e4:47:51:81    False
68:ca:e4:47:51:8e    False
68:ca:e4:47:51:8f    False
68:ca:e4:47:54:21     True
68:ca:e4:47:6e:e0     True
68:ca:e4:48:9d:a0     True
68:ca:e4:48:9d:a1     True
68:ca:e4:48:9d:ae     True
68:ca:e4:48:9d:af     True
88:36:6c:08:80:8c     True
88:36:6c:08:80:8e     True
88:36:6c:a1:47:7a     True
88:36:6c:bb:10:06     True
90:9f:33:a8:6b:b4     True
magnetic_x           False
magnetic_y           False
magnetic_z           False
x                    False
y                    False
dtype: bool

In [32]:
drop_list = [
    '68:ca:e4:46:c1:80',
    '68:ca:e4:46:c1:81',
    '68:ca:e4:47:4d:e0',
    '68:ca:e4:47:4d:e1',
    '68:ca:e4:47:4d:ee',
    '68:ca:e4:47:4d:ef',
    '68:ca:e4:47:4e:a0',
    '68:ca:e4:47:4e:a1',
    '68:ca:e4:47:54:21',
    '68:ca:e4:47:6e:e0',
    '68:ca:e4:48:9d:a0',
    '68:ca:e4:48:9d:a1',
    '68:ca:e4:48:9d:ae',
    '68:ca:e4:48:9d:af',
    '88:36:6c:08:80:8c',
    '88:36:6c:08:80:8e',
    '88:36:6c:a1:47:7a',
    '88:36:6c:bb:10:06',
    '90:9f:33:a8:6b:b4',
    '08:d4:6a:e6:ff:39' # Q8
]
print("will dropped:")
for dd in drop_list:
    print(bssid_ssid[dd])

will dropped:
knu-smart
eduroam
knu-smart
eduroam
eduroam
knu-smart
knu-smart
eduroam
eduroam
knu-smart
knu-smart
eduroam
eduroam
knu-smart
DBLAB_5G
DBLAB
NetLab
iptime
iptime5G
Q8(2018)_7622


In [38]:
fin_df = cut_df.drop(drop_list, axis=1)
fin_df.shape, fin_df.columns

((315, 14),
 Index(['28:c2:dd:70:6f:51', '28:c2:dd:70:70:4b', '28:c2:dd:70:70:4f',
        '28:c2:dd:70:70:51', '28:c2:dd:70:93:0d', '68:ca:e4:47:51:80',
        '68:ca:e4:47:51:81', '68:ca:e4:47:51:8e', '68:ca:e4:47:51:8f',
        'magnetic_x', 'magnetic_y', 'magnetic_z', 'x', 'y'],
       dtype='object'))

In [35]:
print("survived wifi list")
for bb in (set(bbsid_list) - set(drop_list)):
    print(f"{bb}: {bssid_ssid[bb]}")

survived wifi list
28:c2:dd:70:6f:51: SCD-696013949W_FB
68:ca:e4:47:51:80: knu-smart
28:c2:dd:70:70:4b: SCD-696013969W_54
68:ca:e4:47:51:8f: knu-smart
28:c2:dd:70:70:51: SCD-696013929W_XN
68:ca:e4:47:51:81: eduroam
68:ca:e4:47:51:8e: eduroam
28:c2:dd:70:70:4f: SCD-696013899W_44
28:c2:dd:70:93:0d: SCD-696013979W_QS


In [40]:
fin_df.fillna(-100.0, inplace=True)

In [43]:
fin_df.isnull().sum()

28:c2:dd:70:6f:51    0
28:c2:dd:70:70:4b    0
28:c2:dd:70:70:4f    0
28:c2:dd:70:70:51    0
28:c2:dd:70:93:0d    0
68:ca:e4:47:51:80    0
68:ca:e4:47:51:81    0
68:ca:e4:47:51:8e    0
68:ca:e4:47:51:8f    0
magnetic_x           0
magnetic_y           0
magnetic_z           0
x                    0
y                    0
dtype: int64

In [44]:
fin_df.to_csv("./data/cut_off_data.csv")